In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import os
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

2023-04-11 00:13:21.156772: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


1


2023-04-11 00:13:22.183609: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-11 00:13:22.184348: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-11 00:13:22.223291: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 00:13:22.223787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2023-04-11 00:13:22.223880: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-11 00:13:22.229516: I tensorflow/stream_executor/platform/def

In [3]:
## Defining batch specfications
batch_size = 50
img_height = 256
img_width = 256

In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range = 0.2,
)
train_generator = train_datagen.flow_from_directory(
        'dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
        target_size=(img_height,img_width),
        batch_size=batch_size,
        class_mode="sparse",
#         save_to_dir="C:\\Code\\potato-disease-classification\\training\\AugmentedImages"
)

Found 70295 images belonging to 38 classes.


In [4]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator =  valid_datagen.flow_from_directory(
   'dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
    target_size= (img_height, img_width),
    batch_size=batch_size,
    class_mode="sparse"
)

Found 17572 images belonging to 38 classes.


In [11]:
import json
with open('label.json','w') as f:
    json.dump(train_generator.class_indices,f)

In [17]:
file = open('label.json','r')
temp = json.loads(file.read())
temp= list(temp.keys())
temp[0]

'Apple___Apple_scab'

In [5]:
class_names = list(train_generator.class_indices.keys())
class_names

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [6]:
#training_ds = training_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
#validation_ds = validation_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
# Defining Cnn
model = tf.keras.models.Sequential([
    layers.Conv2D(32, 3, activation='relu',input_shape=(256,256,3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),

    layers.Dense(len(class_names), activation= 'softmax')
])

2023-04-09 19:38:37.144278: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 19:38:37.144846: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-09 19:38:37.145029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-09 19:38:37.145213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43Gi

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=3,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [11]:
# lets train our CNN
history = model.fit(train_generator,validation_data=valid_generator,callbacks=callbacks,epochs = 20)

2023-04-09 19:38:50.172022: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-09 19:38:50.172372: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1800500000 Hz


Epoch 1/20


2023-04-09 19:38:50.948216: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-09 19:38:55.017385: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-04-09 19:39:02.245649: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-04-09 19:39:02.279287: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-09 19:39:03.189045: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.42GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-09 19:3

 724/1406 [==============>...............] - ETA: 16:24 - loss: 2.7531 - accuracy: 0.2451

2023-04-09 19:56:34.573802: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-09 19:56:38.848033: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-09 19:56:39.078922: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-09 19:56:39.571112: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Alloc

1406/1406 [==============================] - 2255s 2s/step - loss: 2.2809 - accuracy: 0.3607 - val_loss: 0.5808 - val_accuracy: 0.8137
Epoch 2/20
1406/1406 [==============================] - 1224s 870ms/step - loss: 0.7203 - accuracy: 0.7741 - val_loss: 0.3458 - val_accuracy: 0.8864
Epoch 3/20
1406/1406 [==============================] - 1248s 887ms/step - loss: 0.4691 - accuracy: 0.8512 - val_loss: 0.2617 - val_accuracy: 0.9149
Epoch 4/20
1406/1406 [==============================] - 1244s 885ms/step - loss: 0.3577 - accuracy: 0.8869 - val_loss: 0.4662 - val_accuracy: 0.8572
Epoch 5/20
1406/1406 [==============================] - 1266s 900ms/step - loss: 0.3082 - accuracy: 0.9016 - val_loss: 0.1931 - val_accuracy: 0.9369
Epoch 6/20
1406/1406 [==============================] - 1406s 1s/step - loss: 0.2674 - accuracy: 0.9154 - val_loss: 0.1981 - val_accuracy: 0.9351
Epoch 7/20
1406/1406 [==============================] - 1469s 1s/step - loss: 0.2414 - accuracy: 0.9232 - val_loss: 0.2365 

In [13]:
ok = model.
ok

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [31]:
x,y = next(valid_generator)

In [32]:
np.argmax(model.predict(x),axis=1)

array([14, 10, 16,  3, 21,  4, 37, 20, 27,  8, 20, 36, 12, 23, 22, 36, 11,
       28, 17, 10, 26, 11,  5, 36, 36, 32, 33, 29, 33, 24,  7, 13,  4, 22,
       32,  2, 29,  2, 37,  7, 18, 33,  1, 30, 11, 12,  9, 27, 21,  0])

In [33]:
y

array([14., 10., 16.,  3., 21.,  4., 33., 20., 27.,  8., 20., 36., 12.,
       23., 22., 36., 11., 28., 17., 10., 26., 11.,  5., 36., 36., 32.,
       33., 29., 33., 24.,  7., 13.,  4., 22., 32.,  2., 29.,  2., 37.,
        7., 18., 33.,  1., 30., 11., 12.,  9., 27., 21.,  0.],
      dtype=float32)

In [21]:
AccuracyVector = []
plt.figure(figsize=(30, 30))
for images, labels in next(valid_generator):
    predictions = model.predict(images)
    predlabel = []
    prdlbl = []
    
    for mem in predictions:
        predlabel.append(class_names[np.argmax(mem)])
        prdlbl.append(np.argmax(mem))
    
    AccuracyVector = np.array(prdlbl) == labels
    for i in range(40):
        ax = plt.subplot(10, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title('Pred: '+ predlabel[i]+' actl:'+class_names[labels[i]])
        plt.axis('off')
        plt.grid(True)

ValueError: too many values to unpack (expected 2)

<Figure size 3000x3000 with 0 Axes>

In [17]:
for images, labels in validation_ds.take(1):
    predictions = model.predict(images)

2023-04-08 17:00:05.525008: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 677 of 800
2023-04-08 17:00:07.202953: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:230] Shuffle buffer filled.


In [33]:
for x,y in validation_ds.take(1):
    print(np.max(model.predict(x),axis=1)*100)

[100.        99.92011   99.9995    99.99672   95.1691   100.
  99.78005   99.9997    99.98665   99.96093  100.        86.755684
 100.        99.99993   99.98364   59.693413  99.990555  99.999985
  65.33012  100.        99.99776   99.999916  63.607067  99.982254
 100.        99.554214  99.99988   99.970825  99.897514  93.72962
  99.14273   99.98228   99.71222   58.74198   97.14632   99.29327
 100.        96.243835  99.99701   99.999985 100.       100.
  95.15707   44.151955  99.99969  100.        74.88902   99.99971
  99.98305  100.       100.        78.52572  100.        99.9996
  99.29755   99.99988   99.91606  100.       100.        99.99907
 100.        99.995735  92.51287   99.62885   99.997696  99.94324
  87.902824  58.91414  100.        99.6412    79.764725  99.9975
  99.99998   99.98728   55.624752  99.99958   99.997154  99.55522
  99.99894   99.99557   99.99945   96.16217   99.18517   99.99936
  96.70523  100.        99.999916  99.99994   99.999985  99.99987
  99.99996   59.402

In [41]:
from tensorflow.keras.models import load_model
import cv2 as cv
model = load_model('model2.h5')

In [42]:
path = 'dataset/test/test'
for i in os.listdir(path):
    img = cv.imread(os.path.join(path,i))
    img = img/255
    img = cv.resize(img,(256,256))
    img = img.reshape(1,256,256,3)
    print("actual :"+ i + " Pred : " + class_names[np.argmax(model.predict(img))])
    print("accuracy : ",np.max(model.predict(img)) * 100)

actual :AppleCedarRust1.JPG Pred : Pepper,_bell___healthy
accuracy :  99.00118708610535
actual :AppleCedarRust2.JPG Pred : Cherry_(including_sour)___Powdery_mildew
accuracy :  78.98526787757874
actual :AppleCedarRust3.JPG Pred : Tomato___healthy
accuracy :  45.14709711074829
actual :AppleCedarRust4.JPG Pred : Tomato___healthy
accuracy :  98.34774136543274
actual :AppleScab1.JPG Pred : Pepper,_bell___healthy
accuracy :  51.10576152801514
actual :AppleScab2.JPG Pred : Orange___Haunglongbing_(Citrus_greening)
accuracy :  99.99611377716064
actual :AppleScab3.JPG Pred : Potato___healthy
accuracy :  44.447773694992065
actual :CornCommonRust1.JPG Pred : Corn_(maize)___Common_rust_
accuracy :  99.97511506080627
actual :CornCommonRust2.JPG Pred : Potato___healthy
accuracy :  92.46914982795715
actual :CornCommonRust3.JPG Pred : Corn_(maize)___Common_rust_
accuracy :  99.99313354492188
actual :PotatoEarlyBlight1.JPG Pred : Potato___healthy
accuracy :  75.19046068191528
actual :PotatoEarlyBlight2.

In [60]:
os.listdir(path)

['AppleCedarRust1.JPG',
 'AppleCedarRust2.JPG',
 'AppleCedarRust3.JPG',
 'AppleCedarRust4.JPG',
 'AppleScab1.JPG',
 'AppleScab2.JPG',
 'AppleScab3.JPG',
 'CornCommonRust1.JPG',
 'CornCommonRust2.JPG',
 'CornCommonRust3.JPG',
 'PotatoEarlyBlight1.JPG',
 'PotatoEarlyBlight2.JPG',
 'PotatoEarlyBlight3.JPG',
 'PotatoEarlyBlight4.JPG',
 'PotatoEarlyBlight5.JPG',
 'PotatoHealthy1.JPG',
 'PotatoHealthy2.JPG',
 'TomatoEarlyBlight1.JPG',
 'TomatoEarlyBlight2.JPG',
 'TomatoEarlyBlight3.JPG',
 'TomatoEarlyBlight4.JPG',
 'TomatoEarlyBlight5.JPG',
 'TomatoEarlyBlight6.JPG',
 'TomatoHealthy1.JPG',
 'TomatoHealthy2.JPG',
 'TomatoHealthy3.JPG',
 'TomatoHealthy4.JPG',
 'TomatoYellowCurlVirus1.JPG',
 'TomatoYellowCurlVirus2.JPG',
 'TomatoYellowCurlVirus3.JPG',
 'TomatoYellowCurlVirus4.JPG',
 'TomatoYellowCurlVirus5.JPG',
 'TomatoYellowCurlVirus6.JPG']

In [36]:
img = cv.imread(os.path.join(path,'AppleCedarRust1.JPG'))
img = cv.resize(img,(256,250))

In [40]:
model.save('model2.h5')